In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import geopandas 
from shapely import wkt
import folium
import geodatasets

In [ ]:
cities = pd.read_csv('data/cities.csv')

In [ ]:
cities
cities['Coordinates'] = geopandas.GeoSeries.from_wkt(cities['coords'])
gdf = geopandas.GeoDataFrame(cities, geometry='Coordinates')

In [ ]:
gdf = gdf.set_crs("EPSG:4326")
m = folium.Map(location=[40.70, -73.94], zoom_start=10, tiles="CartoDB positron")

for _, r in gdf.iterrows():
    lat =r["Coordinates"].y
    lon = r['Coordinates'].x
    folium.Marker(
        location=[lat, lon],
    ).add_to(m)

In [ ]:
m.save('Images/map.html')

In [ ]:
lines = pd.read_csv('data/lines.csv')

In [ ]:
lines = lines.rename(columns={'name': 'line_name'})

In [ ]:
lst = ['data/station_lines.csv', 'data/stations.csv', 'data/systems.csv', 'data/track_lines.csv', 'data/tracks.csv']
station_lines = pd.read_csv(lst[0])
stations = pd.read_csv(lst[1])
systems = pd.read_csv(lst[2])
track_lines = pd.read_csv(lst[3])
tracks = pd.read_csv(lst[4])

In [ ]:
tracks

In [ ]:
lines_drop = lines.copy()
lines_drop = lines_drop.drop(columns=['url_name', 'color', 'transport_mode_id'])

In [ ]:
lines_drop

In [ ]:
cities

In [ ]:
cities_drop = cities.copy()
cities_drop = cities_drop.drop(columns='url_name')
cities_usa = cities_drop[cities_drop['country'] == "United States"]

In [ ]:
cities_lines = lines_drop.merge(cities_usa, left_on='city_id', right_on='id', how='left')

In [ ]:
cities_lines

In [ ]:
cities_lines = cities_lines.rename(columns={'name':'city', 'city_id': 'id'}).dropna(subset='city')

In [ ]:
cities_lines

In [ ]:
cities_stations = stations.merge(cities_usa, left_on='city_id', right_on='id', how='left')
cities_stations = cities_stations.rename(columns={'name_y':'city', 'city_id':'id'}).dropna(subset='city')
cities_stations = cities_stations[cities_stations['buildstart']> 0]
cities_stations['buildstart'] = cities_stations['buildstart'].astype(int)

In [ ]:
cities_stations['city'].value_counts().plot.bar()

In [ ]:
#cities_stations = cities_stations[(cities_stations['city'] != 'Chicago') & (cities_stations['city'] != 'Los Angeles')]

In [ ]:
cities_stations

In [ ]:
def cityplot():
    for city in cities_stations['city'].unique():
        print(city)
        ax = cities_stations[cities_stations['city'] == city].groupby('buildstart').size().cumsum().plot(
        x='buildstart', y='0', xlabel='Year', ylabel='Total Stations')
    return ax
ax = cityplot()
ax.set_title('Urban Rail Infrastructure in The United States')
ax.legend(cities_stations['city'].unique())
ax.plot()

In [ ]:
cities_stations['buildstart'].plot.hist(ylabel='Number of Stations Built', xlabel='Year').set_title('Stations Built in the United States')

In [ ]:
ax = sns.histplot(data=cities_stations, x='buildstart', hue='city')
ax.set_xlabel('Year')
ax.set_ylabel('Number of Stations Built')
fig = ax.get_figure()
fig.set_figheight(8)
fig.set_figwidth(8)

In [ ]:
cities_lines

In [ ]:
cities_lines = cities_lines.drop(columns=['id_x', 'id_y', 'system_id'])

In [ ]:
cities_lines_stations = stations.merge(cities_lines, left_on='city_id', right_on='id', how='left')

In [ ]:
cities_lines_stations = cities_lines_stations.dropna(subset='city')

In [ ]:
cities_lines_stations.sort_values(by='city_id')

In [ ]:
cls = cities_stations.rename(columns={'Coordinates': 'City Coordinates'})

In [ ]:
cls['Station Coordinates'] = geopandas.GeoSeries.from_wkt(cls['geometry'])
gdf = geopandas.GeoDataFrame(cls, geometry='Station Coordinates')

In [ ]:
tracks['geometry'] = tracks['geometry'].apply(wkt.loads)

In [ ]:
#tracks['tracks_coords'] = geopandas.GeoSeries.from_wkt(cls['geometry'])

In [ ]:
#tracks = tracks.drop(columns='tracks_coords')


In [ ]:
tracks_gdf = geopandas.GeoDataFrame(tracks)

In [ ]:
tracks_gdf= tracks_gdf.set_crs("EPSG:4326")

In [ ]:
tracks_gdf

In [ ]:
def build_map(gdf, column):
    gdf = gdf.set_crs("EPSG:4326")
    m = folium.Map(location=[40.70, -73.94], zoom_start=10, tiles="CartoDB positron")

    for _, r in gdf.iterrows():
        lat = r[column].y
        lon = r[column].x
        folium.Marker(
            location=[lat, lon],
        ).add_to(m)
   

In [ ]:
def add_lines(gdf, column)
    gdf = gdf.set_crs('EPSG:4326')
    

In [ ]:
build_map(gdf, 'City Coordinates')
build_map(gdf, 'Station Coordinates')
build_map(tracks_gdf, 'geometry')

In [ ]:
m.save('Images/map.html')

In [ ]:
cities_id = cities_usa.rename(columns={'id':'city_id'})

In [ ]:
cities_line = lines_drop.merge(citi)

In [ ]:
cities_line

In [ ]:
stations

In [ ]:
stations_drop = stations.copy()
#stations_usa = stations_drop[stations_drop['country'] == "United States"]
#stations_drop['cities'] = stations_drop[[stations_drop['city_id']== 147]] = 'Chicago'
stations_drop = stations_drop.dropna(subset='buildstart')
stations_drop = stations_drop[stations_drop['buildstart'] == 0] = np.nan

In [ ]:
stations_drop